In [ ]:
import os
import sys
from sys import exit as error
from datetime import datetime
# from time import process_time
# from random import randrange
import time
import warnings
# import inspect
import copy

In [ ]:
import numpy
import scipy

In [ ]:
%logstart -o

In [ ]:
AEMPYX_ROOT = os.environ["AEMPYX_ROOT"]
mypath = [AEMPYX_ROOT+"/aempy/modules/", AEMPYX_ROOT+"/aempy/scripts/"]
for pth in mypath:
    if pth not in sys.path:
        sys.path.insert(0,pth)

In [ ]:
from version import versionstrg

In [ ]:
import aesys
import util
import inverse
import alg

In [ ]:
warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
AEMPYX_DATA = os.environ["AEMPYX_DATA"]

In [ ]:
rng = numpy.random.default_rng()
nan = numpy.nan  # float("NaN")
version, _ = versionstrg()
titstrng = util.print_title(version=version, fname=__file__, out=False)
print(titstrng+"\n\n")

In [ ]:
OutInfo = False
now = datetime.now()

In [ ]:
"""
System related settings.
Data transformation is now allowed with three possible options:
DataTrans   = 0           raw data
            = 1           natural log of data
            = 2           asinh transformation
An error model is applied for the raw data, which is
mixed additive/multiplicative. in case of data transformation,
errors are also transformed.
"""
# AEM_system = "genesis"
AEM_system = "aem05"  # "genesis"
if "aem05" in AEM_system.lower():
    FwdCall,NN, _, _, _, = aesys.get_system_params(System=AEM_system)
    nL = NN[0]
    ParaTrans = 1
    DataTrans = 0
    DatErr_add =  50.
    DatErr_mult = 0.03
    data_active = numpy.ones(NN[2], dtype="int8")

In [ ]:
if "genes" in AEM_system.lower():
    FwdCall, NN, _, _, _, = aesys.get_system_params(System=AEM_system)
    nL = NN[0]
    ParaTrans = 1
    DataTrans = 0
    DatErr_add = 100.
    DatErr_mult = 0.01
    data_active = numpy.ones(NN[2], dtype="int8")
    data_active[0:11]=0  # only vertical component
    # data_active[10:11]=0  # Vertical + 'good' hoizontals'

In [ ]:
"""
input formats are ".npz",".nc4",".asc"
"""
ReverseDir = False

In [ ]:
FileList = "search"  # "search", "read"
InDatDir = AEMPYX_DATA + "/Projects/InvParTest/proc_delete_PLM3s/"
SearchStrng = "*3s_k?.npz"

In [ ]:
if "set" in FileList.lower():
    # InDatDir = AEMPYX_DATA + "/ERT_AEM_Profiles/data/"
    InDatDir ="/home/vrath/work/AEM_Data/Tellus/data/SYNTH/"
    print("Data files read from dir:  %s" % InDatDir)
    dat_files = ["SYNTH_AEM05_1Layer_Resistor.asc",
                "SYNTH_AEM05_1Layer_Conductor.asc",]

In [ ]:
else:
    # how = ["search", SearchStrng, InDatDir]
    # how = ["read", FileList, InDatDir]
    dat_files = util.get_data_list(how=["search", SearchStrng, InDatDir],
                              out= True, sort=True)
    ns = numpy.size(dat_files)

In [ ]:
ns = numpy.size(dat_files)
if ns ==0:
    error("No files set!. Exit.")

In [ ]:
"""
Output formats are ".npz"
"""
OutFileFmt = ".npz"
# OutDatDir = AEMPYX_DATA + "/Nearest/fwd_compare/models/"
OutDatDir = InDatDir+"/results_prior/"
print("Models written to dir: %s " % OutDatDir)

In [ ]:
if not os.path.isdir(OutDatDir):
    print("File: %s does not exist, but will be created" % OutDatDir)
    os.mkdir(OutDatDir)

In [ ]:
"""
Define inversion type  optional additional parameters (e.g., Waveforms )
"""
RunType = "TikhOpt" # "TikhOcc",  "MAP_ParSpace", "MAP_DatSpace","Jack","DoI", "RTO""
Uncert = True

In [ ]:
RegFun = "fix" # "fix", "lcc", "gcv", "mle"
RegVal0 = 1.e-5
NTau0 = 1
Tau0min = numpy.log10(RegVal0)
Tau0max = numpy.log10(RegVal0)
Tau0 = numpy.logspace(Tau0min, Tau0max, NTau0)

In [ ]:
if any(s in RegFun.lower() for s in ["gcv", "upr", "ufc", "mle", "lcc"]):
    RegVal1Min = 0.1
    RegVal1Max = 1000.
    NTau1 =64
    Tau1min = numpy.log10(RegVal1Min)
    Tau1max = numpy.log10(RegVal1Max)
else:
    RegVal1 =0.00001
    NTau1 =1
    Tau1min = numpy.log10(RegVal1)
    Tau1max = numpy.log10(RegVal1)

In [ ]:
Tau1 = numpy.logspace(Tau1min, Tau1max, NTau1)
nreg = NTau0 * NTau1

In [ ]:
NSamples = 100
Percentiles = [10., 20., 30., 40., 50., 60., 70., 80., 90.] # linear
# Percentiles = [2.3, 15.9, 50., 84.1,97.7]                   # 95/68

In [ ]:
"""
Model definition
"""

In [ ]:
SetPrior = "set"
ParaTrans = 1

In [ ]:
Nlyr = 1
dzstart = 3.
dzend = 10.
dz = numpy.logspace(numpy.log10(dzstart), numpy.log10(dzend), Nlyr)
z = numpy.append(0.0, numpy.cumsum(dz))

In [ ]:
zerolayer = numpy.zeros(Nlyr)
izerolayer = zerolayer.astype(int)
onelayer = numpy.ones(Nlyr)
ionelayer = onelayer.astype(int)

In [ ]:
mod_act, mod_apr, mod_var, mod_bnd, m_state = inverse.init_1dmod(Nlyr)

In [ ]:
mod_act[0*Nlyr:1*Nlyr] = ionelayer[0:Nlyr]
sizepar = numpy.shape(mod_act)
mpara = sizepar[0]

In [ ]:
Guess_r = 300.0  # initial guess for resistivity in mod_apr
Guess_s = 0.3   # mod_std defines standard deviation of mod_apr
mod_apr[0*Nlyr:1*Nlyr] = Guess_r * onelayer[0:Nlyr]
mod_var[0*Nlyr:1*Nlyr] = numpy.power(Guess_s * onelayer[0:Nlyr],2)
mod_apr[6*Nlyr:7*Nlyr-1] = dz[0:Nlyr - 1]
mod_var[6*Nlyr:7*Nlyr-1] = numpy.power(1. * onelayer[0:Nlyr-1],2)

In [ ]:
# mod_bnd = mumpy.array([])
max_val = 1.e+30
min_val = 1.e-30
# max_val = mod_apr[mod_act!=0] + 3*mod_std[mod_act!=0]
# mod_bnd[mod_act!=0, 1] = max_val
# min_val = mod_apr[mod_act!=0] - 3*mod_std[mod_act!=0]
# mod_bnd[mod_act!=0, 0] = min_val
mod_bnd[:,0] = min_val
mod_bnd[:,1] = max_val

In [ ]:
if OutInfo:
    #   print \
    #   (" Parameter set for inverting: \n", mod_act)
    print(" Layer thicknesses: \n", dz)
    print(" Layer interface depths: \n", z)
    print(" Initial halfspace resistivity of %6.2f Ohm*m" % (Guess_r))
    print(" Log Standard error of %6.2f " % (Guess_s))
    if not (mod_bnd == None) or (numpy.size(mod_bnd) == 0):
        print(" Upper limits: \n", mod_bnd[:, 1])
        print(" Lower limits: \n", mod_bnd[:, 0])

In [ ]:
"""
Setup Controls for different Algorithms
"""
if "tikhopt" in  RunType.lower():
    """
    Prepare differential operator base methods for regularization matrices
    """
    D0 = inverse.diffops(dz, der=False, mtype="sparse", otype="L0")
    L = [D0 for D in range(7)]
    L0 = scipy.sparse.block_diag(L)
    Cm0 = L0.T@L0
    Cm0 = inverse.extract_cov(Cm0, mod_act)

    D1 = inverse.diffops(dz, der=False, mtype="sparse", otype="L1")
    L = [D1 for D in range(7)]
    L1 = scipy.sparse.block_diag(L)
    Cm1 = L1.T@L1
    Cm1 = inverse.extract_cov(Cm1, mod_act)

    Maxiter = 10
    Maxreduce = 5
    Rfact = 0.66
    LinPars = [Maxreduce, Rfact]


    ThreshRMS = [0.9, 1.0e-2, 1.0e-2]
    Delta = [1.e-5]
    RegShift = 1
    Ctrl = dict([
        ("system", [AEM_system, FwdCall]),
        ("inversion",
         numpy.array([RunType, RegFun, Tau0, Tau1, Maxiter, ThreshRMS,
                      LinPars, SetPrior, Delta, RegShift], dtype=object)),
        ("covar",
         numpy.array([L0, Cm0, L1, Cm1], dtype=object)),
        ("transform",
         [DataTrans, ParaTrans]),
        ("uncert",
         Uncert)
       ])

In [ ]:
if "occ" in RunType.lower():
    """
    Prepare differential operator base methods for regularization matrices
    """
    D0 = inverse.diffops(dz, der=False, mtype="sparse", otype="L0")
    L = [D0 for D in range(7)]
    L0 = scipy.sparse.block_diag(L)
    Cm0 = L0.T@L0
    Cm0 = inverse.extract_cov(Cm0, mod_act)

    D1 = inverse.diffops(dz, der=False, mtype="sparse", otype="L1", variant=Variant)
    L = [D1 for D in range(7)]
    L1 = scipy.sparse.block_diag(L)
    Cm1 = L1.T@L1
    Cm1 = inverse.extract_cov(Cm1, mod_act)

    Maxiter = 10
    Maxreduce = 5
    Rfact = 0.66
    LinPars = [Maxreduce, Rfact]

    Maxiter = 10
    Maxreduce = 5
    Rfact = 0.66
    ThreshRMS = [0.5, 1.0e-2, 1.0e-2]
    L = L1
    TauSeq = [0.5]
    Delta = [1.e-5]
    Ctrl = dict([
        ("system", [AEM_system, FwdCall]),
        ("inversion",
         numpy.array([RunType, TauSeq, Tau0, Maxiter,ThreshRMS,
                      LinPars, SetPrior, Delta],dtype=object)),
        ("covar",
         numpy.array( [L0, Cm0, L1, Cm1], dtype=object)),
        ("transform",
         [DataTrans, ParaTrans]),
        ("uncert",
         Uncert)
       ])

In [ ]:
if "map" in  RunType.lower():

    """
    Prepare explicit covariances for MAP and related methods
    """

    zc = inverse.set_zcenters(dz)
    xc = numpy.zeros_like(zc)
    yc = numpy.zeros_like(zc)
    CorrL = numpy.array([30.0, 30.0, 30.0])

    """
    This setup is a workaround, correct only for rho-only inversion
    """

    mvar  = mod_var[0*Nlyr:1*Nlyr]
    # inverse.extract_mod(mod_var, mod_act)

    if "par"in RunType.lower():
        InvSpace = "par"
        Cmi, CmiS = inverse.covar(xc, yc, zc, covtype= ["exp", CorrL],
                  var=mvar, sparse=False, thresh=0.05, inverse=True)
        Cmi=inverse.extract_cov(Cmi, mod_act)
        Cmi = scipy.sparse.block_diag([Cmi for Cmi in range(7)])
        CmiS=inverse.extract_cov(CmiS, mod_act)
        CmiS = scipy.sparse.block_diag([CmiS for Cmis in range(7)])
        C, sC = Cmi, CmiS
    else:
        InvSpace = "dat"
        Cm, CmS = inverse.covar(xc, yc, zc, covtype= ["exp", CorrL],
                  var=mvar, sparse=False, thresh=0.05, inverse=False)
        Cm=inverse.extract_cov(Cm, mod_act)
        Cm = scipy.sparse.block_diag([Cm for Ci in range(7)])
        CmS=inverse.extract_cov(CmS, mod_act)
        CmS = scipy.sparse.block_diag([CmS for CmS in range(7)])
        C, sC = Cm, CmS

    Maxiter = 10
    Maxreduce = 5
    Rfact = 0.66
    ThreshRMS = [0.5, 1.0e-2, 1.0e-2]
    Delta = [1.e-5]
    TauSeq = [0.5]
    RegShift = 1
    Ctrl = dict([
        ("system",
         [AEM_system, FwdCall]),
        ("inversion",
         numpy.array([RunType, InvSpace, RegFun, Tau0, Tau1, Maxiter,ThreshRMS,
                      LinPars, SetPrior, Delta, RegShift], dtype=object)),
        ("covar",
         numpy.array([C, sC], dtype=object)),
        ("transform",
         [DataTrans, ParaTrans]),
        ("uncert",
         Uncert)
       ])

In [ ]:
if OutInfo:
    print(Ctrl.keys())

In [ ]:
OutStrng = "_halfspace_Results"
print("ID string: Iinput file + %s " % OutStrng)

In [ ]:
fcount =0
for file in dat_files:

    start = time.time()

    fcount=fcount+1

    name, ext = os.path.splitext(file)
    filein = InDatDir+file
    print("\n Reading file " + filein)

    fileout = OutDatDir + name + OutStrng
    numpy.savez_compressed(file=fileout+"_ctrl"+OutFileFmt,**Ctrl)


    DataObs, Header, _ = aesys.read_aempy(File=filein,
                                   System=AEM_system, OutInfo=False)

    fl_name = DataObs[0, 0]
    site_x = DataObs[:, 1]
    site_y = DataObs[:, 2]
    site_gps = DataObs[:, 3]
    site_alt = DataObs[:, 4]
    site_dem = DataObs[:, 5]
    dat_obs =  DataObs[:, 6:6+NN[2]]
    [nsite,ndata] = numpy.shape(dat_obs)


    if "read" in SetPrior.lower():
        error("Prior model read not yet implemented! Exit.")
        # file,filext0 = os.path.splitext(file)
        # tmp = numpy.load(file, allow_pickle=True)
        # mod_aprel = tmp["site_modl"]
        # mod_apr = numpy.mat(inverse.extract_mod(mod_apr,mod_act))
    elif "set" in SetPrior.lower() or "upd" in SetPrior.lower():
        mod_apr = mod_apr.copy()
        mod_ini = mod_apr.copy()
        error_ini = mod_var
    else:
        error("Prior model method "+SetPrior.lower()+" not yet implemented! Exit.")



    start = time.time()
    """
    Loop over sites
    """
    sequence = range(nsite)
    if ReverseDir:
        sites = sequence[::-1]
    else:
        sites = sequence


    logsize = (2 + 7*Maxiter)
    site_log = numpy.full((len(sites),logsize), numpy.nan)

    for ii in sites:
        print("\n Invert site #"+str(ii))

        """
        Setup model-related paramter
        """


        if "upd" in SetPrior:
            if ii == 0:
                mod_ini = mod_apr.copy()
                model = mod_ini.copy()
            else:
                mod_ini = model.copy()
                model = mod_ini.copy()

        Model = dict([
            ("m_act", mod_act),
            ("m_apr", mod_apr),
            ("m_var", mod_var),
            ("m_bnd", mod_bnd),
            ("m_ini", mod_ini)
            ])

        """
        Setup data-related paramter
        """
        dat_act = numpy.tile(data_active,(nsite,1))
        dat_err = numpy.zeros_like(dat_obs)
        dat_err[ii, :], _ = inverse.set_errors(dat_obs[ii, :],
                                                daterr_add=DatErr_add,
                                                daterr_mult=DatErr_mult)

        Data = dict([
            ("d_act", dat_act[ii,:]),
            ("d_obs", dat_obs[ii,:]),
            ("d_err", dat_err[ii,:]),
            ("alt", site_alt[ii])
            ])

        """
        Call inversion algorithms
        """
        if "opt" in RunType.lower():
            Results =\
                alg.run_tikh_opt(Ctrl=Ctrl, Model=Model, Data=Data,
                                  OutInfo=OutInfo)

        if "occ" in RunType.lower():
            Results =\
                alg.run_tikh_occ(Ctrl=Ctrl, Model=Model, Data=Data,
                                  OutInfo=OutInfo)

        if "map" in RunType.lower():
            Results =\
                alg.run_map(Ctrl=Ctrl, Model=Model, Data=Data,
                                  OutInfo=OutInfo)


        """
        Store inversion Results
        """
        if OutInfo:
            print("Results: ",Results.keys())


        M = Results["model"]
        D = Results["data"]
        C = Results["log"]

        if ii==0:
            site_num  = numpy.array([ii])
            site_nrms = C[2]
            site_modl = M[0]
            site_merr = M[1]
            site_sens = M[2]
            site_dobs = D[0].reshape((1,-1))
            site_dcal = D[1].reshape((1,-1))
            site_derr = D[2].reshape((1,-1))
            # print("ii=0")
            cc = numpy.hstack((C[0], C[1],
                              C[2],
                              C[3].ravel(),
                              C[4].ravel(),
                              C[5].ravel(),
                              C[6].ravel()))
            site_log[ii,0:len(cc)] = cc
            if Uncert:
                jd = Results["jacd"]
                site_jacd = jd.reshape((1,numpy.size(jd)))
                postcov = Results["cpost"]
                site_postcov = postcov.reshape((1,numpy.size(postcov)))
                mres = Results["mresm"][0]
                site_nump = numpy.sum(numpy.diag(mres))

        else:
           site_num = numpy.vstack((site_num, ii))
           site_nrms = numpy.vstack((site_nrms, C[2]))
           site_modl = numpy.vstack((site_modl, M[0]))
           site_merr = numpy.vstack((site_merr, M[1]))
           site_sens = numpy.vstack((site_sens, M[2]))
           site_dobs = numpy.vstack((site_dobs, D[0]))
           site_dcal = numpy.vstack((site_dcal, D[1]))
           site_derr = numpy.vstack((site_derr, D[2]))
           cc = numpy.hstack((C[0], C[1],
                              C[2],
                              C[3].ravel(),
                              C[4].ravel(),
                              C[5].ravel(),
                              C[6].ravel()))
           site_log[ii,0:len(cc)] = cc

           if Uncert:
               jd = Results["jacd"]
               # print(numpy.shape(site_jacd))
               site_jacd = numpy.vstack((site_jacd,
                                         jd.reshape((1,numpy.size(jd)))))
               postcov = Results["cpost"]
               site_postcov = numpy.vstack((site_postcov ,
                                            postcov.reshape((1,numpy.size(postcov)))))
               mres = Results["mresm"][0]
               site_nump = numpy.vstack((site_nump, numpy.sum(numpy.diag(mres))))



    header=numpy.array(Header, dtype=object),
    fileout = OutDatDir + name + OutStrng + OutFileFmt

    numpy.savez_compressed(
        file=fileout,
        fl_data=file,
        fl_name=fl_name,
        header=header,
        aem_system=AEM_system,
        site_log =site_log,
        mod_ref=mod_apr,
        mod_act=mod_act,
        dat_act=dat_act,
        site_modl=site_modl,
        site_sens=site_sens,
        site_merr=site_merr,
        site_dobs=site_dobs,
        site_dcal=site_dcal,
        site_derr=site_derr,
        site_nrms=site_nrms,
        site_num=site_num,
        site_y=site_y,
        site_x=site_x,
        site_gps=site_gps,
        site_alt=site_alt,
        site_dem=site_dem)

    if Uncert:
        numpy.savez_compressed(
            file=fileout,
            fl_data=file,
            fl_name=fl_name,
            header=header,
            aem_system=AEM_system,
            site_log =site_log,
            mod_ref=mod_apr,
            mod_act=mod_act,
            dat_act=dat_act,
            site_modl=site_modl,
            site_sens=site_sens,
            site_merr=site_merr,
            site_dobs=site_dobs,
            site_dcal=site_dcal,
            site_derr=site_derr,
            site_nrms=site_nrms,
            site_num=site_num,
            site_y=site_y,
            site_x=site_x,
            site_gps=site_gps,
            site_alt=site_alt,
            site_dem=site_dem,
            site_jac= site_jacd,
            site_cov= site_postcov,
            site_nump=site_nump)


    print("\n\nResults stored to "+fileout)
    elapsed = (time.time() - start)
    print (" Used %7.4f sec for %6i sites" % (elapsed, ii+1))
    print (" Average %7.4f sec/site\n" % (elapsed/(ii+1)))

In [ ]:
print("\n\nAll done!")